# Machine Learning Foundation

## Course 6, Part h: Survival Modeling DEMO

## Learning Outcomes

You should walk away from this demonstration with:

1. An understanding of how to approach Survival Analysis.
2. Knowing how to prepare Kaplan-Meier curves for various subgroups.
3. Run a basic Cox survival regression 

__Installation notes:__
This demo uses Python's [lifelines](https://lifelines.readthedocs.io/en/latest/) package. It can be installed using the following command:

```conda install -c conda-forge lifelines```



In [1]:
# Setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter, CoxPHFitter 
import warnings
import os
os.chdir('data')
from colorsetup import colors, palette
sns.set_palette(palette)

ModuleNotFoundError: No module named 'lifelines'

---

# Section 1: Import and Explore Relevant Data

We will use the Churn Dataset described in the lecture for this demonstration. Let's start by reading in the data:

In [ ]:
df = pd.read_pickle('churndata.pkl')
df

Here, we can see various categories of customer variables, as well as `churn_value` (an indicator for whether the customer has left the sample), and `months` (the time variable). Let's start with some simple EDA related to these variables:

In [ ]:
plt.figure(figsize=(8,8))
sns.barplot(data=df, x='churn_value', y='months')


Clearly the average tenure is shorter for those that have churned. As we are interested in understanding the time dimension (how long someone will remain a customer), it is important to understand that our data are censored, i.e. the number of months before churn is biased downwards for customers who have not yet churned.

# Section 2: Plotting the Kaplan-Meier Curve

The Kaplan-Meier Curve represents a simple non-parametric visualization of survival likelihood function in our data. In that sense, it can be thought of as part of the EDA process for survival analysis. Let's start by fitting a simple Kaplan-Meier Curve on our data.

In [ ]:
kmf = KaplanMeierFitter()

kmf.fit(df.months, df.churn_value, label = 'Kaplan Meier Estimate, full sample')

kmf.plot(linewidth=4, figsize=(12, 6))
plt.title('Customer Churn: Kaplan-Meier Curve')
plt.xlabel('Months')
plt.ylabel('Survival probability')

# Section 3: Examining Variables

As we continue to examine the survival function, we may want to relate survival risk with features, or characteristics of our customers. In this example, we will look at diffences in survival risk for customers who have only a single service, vs. customers with multiple services. We can start by plotting a simple histogram for each category, looking at differences between churned and not-churned subsamples.

In [ ]:
# Let's look at the 'multiple services variable'
df1 = df[df.multiple=='Yes']
df2 = df[df.multiple=='No']
fig, ax = plt.subplots(1,2, figsize=(16,8))
df1.groupby('churn_value')['months'].plot(kind='hist', ax=ax[0], title='Customers with Multiple Services.')
ax[0].legend(labels=['Not churned', 'Churned'])
df2.groupby('churn_value')['months'].plot(kind='hist', ax=ax[1], title='Customers with Single Service.')


We can see differences across these groups, let's examing Kaplan-Meier curves for various subsamples. This involves fitting the Kaplan Meier estimator separately for each subsample. 

In [ ]:
kmf.fit(df1.months, df1.churn_value)
kmf.plot(label='Multiple Services', figsize=(12, 6))
kmf.fit(df2.months, df2.churn_value)
kmf.plot(label='Single Service')
plt.title('Number of Services and Churn: Kaplan-Meier Curve')
plt.xlabel('Months')
plt.ylabel('Survival probability')


# Section 4: Cox Proportional Hazards Model

Having looked at our data and related Kaplan-Meier curves, we can formalize the analysis by running survival regression. There are several available models here, documented in the Python [lifelines](https://lifelines.readthedocs.io/en/latest/) module. As we discussed in the course, the Cox model estimates a baseline hazard rate, and assumes features impact this hazard rate proportionally. While this is a strong assumption (and may not be true in general), the model still provides insight into the role of features and their importance impacting survival risk. Let's begin by setting data up for the Cox survival regression.

In [ ]:
# Setting up the data
dfu = df[['multiple', 'churn_value']]
dfd = pd.get_dummies(dfu, drop_first=True)
dfd['months'] = df.months
dfd.rename(columns={'multiple_Yes':'Multiple Services'}, inplace=True)

Now, let's fit a Cox proportional hazard model using the Multiple Services variable.

In [ ]:
cph = CoxPHFitter()
cph.fit(dfd, duration_col='months', event_col='churn_value')
cph.print_summary(style='ascii')

Here, we see regression output for the estimated cox model. The Cox model objet also allows us to plot coefficient estimates to assess their significance, using the `plot` method.

In [ ]:
cph.plot()

Here, we see the coefficient estimate, along with error bars used to assess statistical significance. The result is significantly different from zero, indicating a negative effect of the Multiple Services variable in our analysis.

The Cox model object also allows us to assess results visually, by plotting resulting estimates of survival risk by covariate groups, using the `plot_covariate_groups` method. This can be useful to see estimated changes in survival risk across groups.

In [ ]:
cph.plot_covariate_groups('Multiple Services', [1, 0], plot_baseline=False, figsize=(10, 6), lw=4)

To compare the impact across variables, let's include a few more variables: Satisfaction, Security Service, Backup Service, and Support, and re-run our analysis. We begin by setting up the data, as before.

In [ ]:
# Including additional variables, data setup
dfu = df[['churn_value', 'satisfaction', 'security', 'backup', 'support']]
dfd = pd.get_dummies(dfu, drop_first=True)
dfd['months'] = df.months
#dfd.rename(columns={'multiple_Yes':'Multiple Services'}, inplace=True)
dfd.rename(columns={'backup_Yes':'Backup Service'}, inplace=True)
dfd.rename(columns={'security_Yes':'Security Service'}, inplace=True)
dfd.rename(columns={'support_Yes':'Support Service'}, inplace=True)

In [ ]:
# Fitting Cox Proportional Model
cph = CoxPHFitter()
cph.fit(dfd, duration_col='months', event_col='churn_value')
cph.print_summary(style='ascii')

Again, we see results of regression output, denoting significance levels for each variable, as well as information about our dataset and model fit. 
As above, let's plot coefficients for different variables. This allows us to compare magnitudes and assess which variables have a larger influence on survival risk. 

In [ ]:
cph.plot()

Here, we see coefficient estimates for several variables, which helps us understand both their significance, as well as their relative magnitudes. The Satisfaction variable seems to have the largest impact on survival risk.

Let's look at results for the Satisfaction variable by covariate group. This allows us to plot estiamted survival rates for different values of the Satisfaction and Security variables.

In [ ]:
cph.plot_covariate_groups('satisfaction', [5, 4, 3, 2, 1], plot_baseline=False, figsize=(10, 6), lw=4) 

Our estimated survival functions differ substantially based on this variable (we knew from the previous chart that this was likely the case).

# Summary
In this notebook, we have covered: 

1. How to approach Survival Analysis.
2. Preparing Kaplan-Meier curves for various subgroups.
3. Runing a basic Cox survival regression. 

---
### Machine Learning Foundation (C) 2020 IBM Corporation